In [1]:
source("source.R")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Parsed with column specification:
cols(
  stars = col_integer(),
  name = col_character(),
  text = col_character(),
  date = col_date(format = ""),
  city = col_character(),
  longitude = col_double(),
  latitude = col_double(),
  categories = col_character()
)


# STAT 628 Module 2 Report

_Yanqi Huang_ _Huiyu Jiang_ _Xinjie Ye_

## Introduction

**_Yelp_** is an American corporation headquartered in San Francisco. The **_Yelp.com_** and the **_Yelp_** mobile app, which pubish customers' reviews about local businesses and provide online reservation service, are owned by the corporation. Our _Yelp_ dataset is a part of the **_Yelp Dataset Challenge_**. Our project explores the relationship between text, rate and other attributes of a review based on the dataset.

## Background of Yelp Dataset

The Yelp dataset we used in our project is derived from **_Yelp Dataset Challenge_**, which encourages students to conduct research and analysis on its dataset. The challenge usually contains photo classification, natural language processing & sentiments a analysis and graph mining. 

Our dataset consisting of 9 attributes has about 1.5 million records. The 9 attributes are stars of the review, ID number for Kaggle, name of the business, review text, date of the review, city in which the business is located, longitude , latitude and business categories. 

Our project aims at exploring the sentiments contained by reviews and developing a reasonable and practical model to predict stars given by customers.

## Analysis of Raw Data

There are 1,546,379 records in the dataset. 

In [5]:
train %>% group_by(stars) %>% summarise("Number of Obervations"=n())

stars,Number of Obervations
1,164676
2,152401
3,225710
4,443599
5,559993
